In [2]:
from ast import literal_eval
import pandas as pd
import numpy as np
import warnings
warnings.filterwarnings(action='ignore')
books = pd.read_csv('/content/drive/MyDrive/nlbooks.csv', encoding='utf-8')
books = books.replace({np.nan: 'none'})

books_df=books[['title','description']]

from sklearn.metrics.pairwise import cosine_similarity
from sklearn.feature_extraction.text import CountVectorizer

cnt_vect = CountVectorizer(min_df=0, ngram_range=(1,2))
# fit_transform안에 데이터프레임형태로 넣어주면 안됨. 하나의 변수씩만 넣어주자!
title_vect = cnt_vect.fit_transform(books['title'])
des_vect = cnt_vect.fit_transform(books['description'])
books['book_info']=books['title']+books['description']
b_category=books['kdc_class_no']

# 책 제목에 따른 코사인 유사도 추출
#title_sim = cosine_similarity(title_vect, title_vect)
#des_sim=cosine_similarity(des_vect, des_vect)
#mix_sim=title_sim + des_sim
#print(mix_sim[:6])
#print(title_sim[:6])

1. **Hugging face**

In [3]:
model_id = "sentence-transformers/all-MiniLM-L6-v2"
hf_token = "get your token in http://hf.co/settings/tokens"

In [4]:
%%capture
!pip install retry

In [5]:
import requests
from retry import retry

api_url = f"https://api-inference.huggingface.co/pipeline/feature-extraction/{model_id}"
headers = {"Authorization": f"Bearer {hf_token}"}

In [6]:
@retry(tries=3, delay=10)
def query(texts):
    response = requests.post(api_url, headers=headers, json={"inputs": texts})
    result = response.json()
    if isinstance(result, list):
      return result
    elif list(result.keys())[0] == "error":
      raise RuntimeError(
          "The model is currently loading, please re-run the query."
          )

In [7]:
books_json=pd.DataFrame(books_df).to_json()

In [9]:
import json
bookdata_json = json.loads(books_json, encoding="utf-8")

In [11]:
output = query(bookdata_json)

JSONDecodeError: ignored

In [ ]:
result.to_json()

**2. BERT EMBEDDING**

In [ ]:
!pip install transformers

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
import torch
from transformers import BertTokenizer, BertModel

# OPTIONAL: if you want to have more information on what's happening, activate the logger as follows
import logging
#logging.basicConfig(level=logging.INFO)

import matplotlib.pyplot as plt
%matplotlib inline

# Load pre-trained model tokenizer (vocabulary)
tokenizer = BertTokenizer.from_pretrained('bert-base-multilingual-cased')

Downloading:   0%|          | 0.00/996k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/29.0 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/625 [00:00<?, ?B/s]

In [ ]:
text = "임베딩을 시도할 문장이다."
marked_text = "[CLS] " + text + " [SEP]"

# Tokenize our sentence with the BERT tokenizer.
tokenized_text = tokenizer.tokenize(marked_text)

# Print out the tokens.
print(tokenized_text)

['[CLS]', '임', '##베', '##딩', '##을', '시', '##도', '##할', '문', '##장이', '##다', '.', '[SEP]']


In [ ]:
list(tokenizer.vocab.keys())[10000:10020]

['횟',
 '횡',
 '효',
 '후',
 '훈',
 '훌',
 '훔',
 '훗',
 '훨',
 '훼',
 '휘',
 '휠',
 '휩',
 '휴',
 '흉',
 '흐',
 '흑',
 '흔',
 '흘',
 '흙']

In [ ]:
# Define a new example sentence with multiple meanings of the word "bank"
text = "배를 타고 여행을 간다." \
       "추석에 먹은 배가 맛있었다."

# Add the special tokens.
marked_text = "[CLS] " + text + " [SEP]"

# Split the sentence into tokens.
tokenized_text = tokenizer.tokenize(marked_text)

# Map the token strings to their vocabulary indeces.
indexed_tokens = tokenizer.convert_tokens_to_ids(tokenized_text)

# Display the words with their indeces.
for tup in zip(tokenized_text, indexed_tokens):
    print('{:<12} {:>6,}'.format(tup[0], tup[1]))

[CLS]           101
배             9,330
##를          11,513
타             9,845
##고          11,664
여             9,565
##행을         88,904
간             8,845
##다          11,903
.               119
추             9,765
##석          40,958
##에          10,530
먹             9,266
##은          10,892
배             9,330
##가          11,287
맛             9,254
##있          119,192
##었다         17,706
.               119
[SEP]           102


In [ ]:
segments_ids = [1] * len(tokenized_text)

print (segments_ids)

[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]


In [ ]:
# Convert inputs to PyTorch tensors
tokens_tensor = torch.tensor([indexed_tokens])
segments_tensors = torch.tensor([segments_ids])

In [ ]:
# Load pre-trained model (weights)
model = BertModel.from_pretrained('bert-base-uncased',
                                  output_hidden_states = True, # Whether the model returns all hidden-states.
                                  )

# Put the model in "evaluation" mode, meaning feed-forward operation.
model.eval()


Downloading:   0%|          | 0.00/570 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/440M [00:00<?, ?B/s]

Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertModel: ['cls.seq_relationship.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.decoder.weight', 'cls.seq_relationship.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.bias', 'cls.predictions.transform.dense.bias', 'cls.predictions.transform.dense.weight']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


BertModel(
  (embeddings): BertEmbeddings(
    (word_embeddings): Embedding(30522, 768, padding_idx=0)
    (position_embeddings): Embedding(512, 768)
    (token_type_embeddings): Embedding(2, 768)
    (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
    (dropout): Dropout(p=0.1, inplace=False)
  )
  (encoder): BertEncoder(
    (layer): ModuleList(
      (0): BertLayer(
        (attention): BertAttention(
          (self): BertSelfAttention(
            (query): Linear(in_features=768, out_features=768, bias=True)
            (key): Linear(in_features=768, out_features=768, bias=True)
            (value): Linear(in_features=768, out_features=768, bias=True)
            (dropout): Dropout(p=0.1, inplace=False)
          )
          (output): BertSelfOutput(
            (dense): Linear(in_features=768, out_features=768, bias=True)
            (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
            (dropout): Dropout(p=0.1, inplace=False)
          

In [ ]:
model??
outputs??

In [ ]:
# Run the text through BERT, and collect all of the hidden states produced
# from all 12 layers.
import numpy as np
with torch.no_grad():

    outputs = model(tokens_tensor, segments_tensors)

    # Evaluating the model will return a different number of objects based on 
    # how it's  configured in the `from_pretrained` call earlier. In this case, 
    # becase we set `output_hidden_states = True`, the third item will be the 
    # hidden states from all layers. See the documentation for more details:
    # https://huggingface.co/transformers/model_doc/bert.html#bertmodel
    hidden_states = outputs[2]



IndexError: ignored

In [ ]:
print ("Number of layers:", len(hidden_states), "  (initial embeddings + 12 BERT layers)")
layer_i = 0

print ("Number of batches:", len(hidden_states[layer_i]))
batch_i = 0

print ("Number of tokens:", len(hidden_states[layer_i][batch_i]))
token_i = 0

print ("Number of hidden units:", len(hidden_states[layer_i][batch_i][token_i]))

NameError: ignored

In [ ]:
# For the 5th token in our sentence, select its feature values from layer 5.
token_i = 5
layer_i = 5
vec = hidden_states[layer_i][batch_i][token_i]

# Plot the values as a histogram to show their distribution.
plt.figure(figsize=(10,10))
plt.hist(vec, bins=200)
plt.show()

In [ ]:
# `hidden_states` is a Python list.
print('      Type of hidden_states: ', type(hidden_states))

# Each layer in the list is a torch tensor.
print('Tensor shape for each layer: ', hidden_states[0].size())

In [ ]:
# Concatenate the tensors for all layers. We use `stack` here to
# create a new dimension in the tensor.
token_embeddings = torch.stack(hidden_states, dim=0)

token_embeddings.size()

In [ ]:
# Remove dimension 1, the "batches".
token_embeddings = torch.squeeze(token_embeddings, dim=1)

token_embeddings.size()

In [ ]:
!pip install transformers

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 5.5 MB 4.1 MB/s 
     |████████████████████████████████| 163 kB 64.3 MB/s 
     |████████████████████████████████| 7.6 MB 44.9 MB/s 


In [ ]:
import torch
from transformers import BertTokenizer, BertModel

In [ ]:
model=BertModel.from_pretrained('bert-base-multilingual-cased')

Downloading:   0%|          | 0.00/714M [00:00<?, ?B/s]

Some weights of the model checkpoint at bert-base-multilingual-cased were not used when initializing BertModel: ['cls.predictions.transform.dense.bias', 'cls.predictions.bias', 'cls.seq_relationship.bias', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.decoder.weight', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.LayerNorm.weight', 'cls.seq_relationship.weight']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [ ]:
from transformers import TFAutoModel, AutoTokenizer
# model=TFAutoModel.from_pretrained("<model-name>")
# tokenizer=AutoTokenizer.from_pretrained("<model-name>")

# Store the model we want to use
MODEL_NAME="bert-base-multilingual-cased"

# We need to create the model and tokenizer
model=AutoModel.from_pretrained(MODEL_NAME)
tokenizer=AutoTokenizer.from_pretrained(MODEL_NAME)

NameError: ignored

In [ ]:
tokenized_input_text=tokenizer(text, return_tensors="pt")
for key, value in tokenized_input_text.items():
    print("{}:\n\t{}".format(key,value))

tokenized_text=tokenizer.tokenize(text)
print(tokenized_text)
input_ids=tokenizer.encode(text)
print(input_ids)
decoded_ids=tokenizer.decode(input_ids)
print(decoded_ids)  

In [ ]:
# Bert Embedding

In [ ]:
pip install sentence-transformers

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
sen = [
    "Three years later, the coffin was still full of Jello.",
    "The fish dreamed of escaping the fishbowl and into the toilet where he saw his friend go.",
    "The person box was packed with jelly many dozens of months later.",
    "He found a leprechaun in his walnut shell."
]
from sentence_transformers import SentenceTransformer
model = SentenceTransformer('bert-base-nli-mean-tokens')
#Encoding:
sen_embeddings = model.encode(sen)
sen_embeddings.shape

(4, 768)

In [ ]:
from sklearn.metrics.pairwise import cosine_similarity
#let's calculate cosine similarity for sentence 0:
cosine_similarity(
    [sen_embeddings[0]],
    sen_embeddings[1:]
)

array([[0.33088914, 0.7219258 , 0.5548363 ]], dtype=float32)

In [ ]:
model = SentenceTransformer('bert-base-nli-mean-tokens')
#Encoding:
sen_embeddings = model.encode()
sen_embeddings.shape

TypeError: ignored

In [ ]:
from ast import literal_eval
import pandas as pd
import numpy as np
import warnings
warnings.filterwarnings(action='ignore')
books = pd.read_csv('/content/drive/MyDrive/nlbooks.csv', encoding='utf-8')
books = books.replace({np.nan: 'none'})

books_df=books[['title','description']]

#books_df[['description']=='none']=None
books_df_nax=books_df.dropna(axis='rows')



from sklearn.metrics.pairwise import cosine_similarity
from sklearn.feature_extraction.text import CountVectorizer

#cnt_vect = CountVectorizer(min_df=0, ngram_range=(1,2))
# fit_transform안에 데이터프레임형태로 넣어주면 안됨. 하나의 변수씩만 넣어주자!
#title_vect = cnt_vect.fit_transform(books['title'])
#des_vect = cnt_vect.fit_transform(books['description'])
#books['book_info']=books['title']+books['description']
#b_category=books['kdc_class_no']

# 책 제목에 따른 코사인 유사도 추출
#title_sim = cosine_similarity(title_vect, title_vect)
#des_sim=cosine_similarity(des_vect, des_vect)
#mix_sim=title_sim + des_sim
#print(mix_sim[:6])
#print(title_sim[:6])

In [ ]:
books_df_nax

,title,description
0,너에게 목소리를 보낼게 - &lt;달빛천사&gt; 성우 이용신의 첫 번째 에세이,2004년 방영한 애니메이션 &lt;달빛천사&gt;에서 주인공 루나(풀문) 역을 맡...
1,일기에도 거짓말을 쓰는 사람 - 99년생 시인의 자의식 과잉 에세이,“그러니 나는 말하고 싶은 것을 말하겠다”「침착하게 사랑하기」 차도하 시인 첫 에세...
2,본격 한중일 세계사 12 - 임오군란과 통킹 위기,한중일 관계의 결정적 분기점인 임오군란의 막전 막후를 다룬다. 러시아의 세력 확장을...
3,즉시 기분을 바꿔드립니다 - 신기하게 마음이 편해지는 응급 처방전,"누구에게나 기분 구급상자가 필요하다. 하나씩 하나씩 차근차근, 좋은 기분을 쌓고 건..."
4,"오늘도 리추얼 : 음악, 나에게 선물하는 시간",나다운 일상을 만드는 사람들의 이야기를 담은 『오늘도 리추얼』 시리즈가 위즈덤하우스...
...,...,...
157113,설민석의 세계사 대모험 11 (영국 산업 혁명 편: 혁명에 맞선 자들),none
157114,A Clinical Atlas: Variations of the Bones of t...,none
157115,"과탑이 알려 주는 대학 생활 한 권으로 끝내기 (어서와, 대학은 처음이지?)",none
157116,중학영문법 총정리 한권으로 끝내기 (EBS강사 혼공쌤 허준석의 &quot;쉽고 재미...,none


In [ ]:
pip install sentence_transformers

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 85 kB 5.0 MB/s 
     |████████████████████████████████| 5.5 MB 54.4 MB/s 
     |████████████████████████████████| 1.3 MB 55.2 MB/s 
     |████████████████████████████████| 163 kB 79.5 MB/s 
     |████████████████████████████████| 7.6 MB 64.4 MB/s 
  Created wheel for sentence-transformers: filename=sentence_transformers-2.2.2-py3-none-any.whl size=125938 sha256=597b0317367a95ec7ad00b219d33f838b590ba9ddcdbdf2cbf63944cce38c9f0
  Stored in directory: /root/.cache/pip/wheels/bf/06/fb/d59c1e5bd1dac7f6cf61ec0036cc3a10ab8fecaa6b2c3d3ee9
Successfully built sentence-transformers


In [ ]:
from sentence_transformers import SentenceTransformer
model = SentenceTransformer('bert-base-nli-mean-tokens')
#Encoding:
sen_embeddings = model.encode(books_df_nax['description'])
sen_embeddings.shape

Downloading:   0%|          | 0.00/391 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/190 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/3.95k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/2.00 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/625 [00:00<?, ?B/s]

...

Downloading:   0%|          | 0.00/466k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/399 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/232k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/229 [00:00<?, ?B/s]

KeyboardInterrupt: ignored

In [ ]:
from random import *

user_books=['A','B','C','D']

i=randint(0,len(user_books))

bindex=books_df_nax(books_df_nax['title']==user_books[i]).index


In [ ]:
from sklearn.metrics.pairwise import cosine_similarity
#let's calculate cosine similarity for sentence 0:
cosine_similarity(
    [sen_embeddings['인덱스']],
    sen_embeddings[1:]
)